In [1]:
import pandas as pd
import numpy as np
import CococoNet_reader
import Name_resolver
import os
import itertools
import warnings
import pickle
import tqdm
warnings.simplefilter(action='ignore', category=FutureWarning)


In [108]:
coconet_species_one = pickle.load( open( "/data/passala/Generated_Tables/Comparing_all_orthogorups_across_plants/Orthogroup_based_coconets/rice_jp_merged_cococonet_pickle.p", "rb" ) )
coconet_species_two = pickle.load( open( "/data/passala/Generated_Tables/Comparing_all_orthogorups_across_plants/Orthogroup_based_coconets/soybean_merged_cococonet_pickle.p", "rb" ) )

In [109]:

common_name_1 = Name_resolver.species_name_resolver('rice_jp','common')
common_name_2 = Name_resolver.species_name_resolver('soybean','common')



In [121]:
def generate_sc_table_formatted(coconet_species_one,coconet_species_two, common_name_1,common_name_2):


    
    trimmed_species_1_cococonet = coconet_species_one[coconet_species_one.columns.intersection(coconet_species_two.columns)]
    trimmed_species_1_cococonet = trimmed_species_1_cococonet[trimmed_species_1_cococonet.index.isin(coconet_species_two.columns)]
    double_species_1_trimmed_cococonet = trimmed_species_1_cococonet.replace(1,0)

    trimmed_species_2_cococonet = coconet_species_two[coconet_species_two.columns.intersection(coconet_species_one.columns)]
    trimmed_species_2_cococonet = trimmed_species_2_cococonet[trimmed_species_2_cococonet.index.isin(coconet_species_one.columns)]
    double_species_2_trimmed_cococonet = trimmed_species_2_cococonet.replace(1,0)
    cococonet_1_set_to_zero = double_species_1_trimmed_cococonet

    cococonet_2_set_to_zero = double_species_2_trimmed_cococonet

    top_10_cococonet_1genes = np.array(
        [cococonet_1_set_to_zero[c].nlargest(10).index.values for c in cococonet_1_set_to_zero]
    )  # using pair list above, cut down top 10 list to relevant genes, probably by adding list as a column in panda and then filtering panda to index of pair list
    top_10_ccn_1_genes_dataframe = pd.DataFrame(
        data=top_10_cococonet_1genes,
        index=cococonet_1_set_to_zero.index,
        columns=[
            "One",
            "Two",
            "Three",
            "Four",
            "Five",
            "Six",
            "Seven",
            "Eight",
            "Nine",
            "Ten",
        ],
    )

    tidy_top_10 = top_10_ccn_1_genes_dataframe.melt(ignore_index= False)

    zipped_pairs = zip(tidy_top_10.index.to_list(),tidy_top_10['value'].to_list())
    binary_masked_cococonet_1 = pd.DataFrame(data = 0, columns = cococonet_1_set_to_zero.columns, index = cococonet_1_set_to_zero.index)
    #binary_masked_cococonet.loc[zip(tuple(tidy_top_10.index.to_list()),tuple(tidy_top_10['value'].to_list()))] = 1
    #binary_masked_cococonet.sum(axis =0)
    for row,column in zipped_pairs:
        binary_masked_cococonet_1.at[row,column] = 1

    ranked_columns_cococonet_2 = double_species_2_trimmed_cococonet.rank()

    dot_product_cococonet_2 = binary_masked_cococonet_1.dot(ranked_columns_cococonet_2)
    subtract_minimum_2 = dot_product_cococonet_2-65 # This is 11+10+9+8+7+6+5+4+3+2


    function_conservations_scores_2 = subtract_minimum_2/(subtract_minimum_2.max().max() - subtract_minimum_2.min().min())
    fc_scores_ranked_2=function_conservations_scores_2.rank()
    sc_scores_2 = fc_scores_ranked_2/fc_scores_ranked_2.shape[0]




    top_10_cococonet_2genes = np.array(
        [cococonet_2_set_to_zero[c].nlargest(10).index.values for c in cococonet_2_set_to_zero]
    )  # using pair list above, cut down top 10 list to relevant genes, probably by adding list as a column in panda and then filtering panda to index of pair list
    top_10_ccn_2_genes_dataframe = pd.DataFrame(
        data=top_10_cococonet_2genes,
        index=cococonet_2_set_to_zero.index,
        columns=[
            "One",
            "Two",
            "Three",
            "Four",
            "Five",
            "Six",
            "Seven",
            "Eight",
            "Nine",
            "Ten",
        ],
    )

    tidy_top_10 = top_10_ccn_2_genes_dataframe.melt(ignore_index= False)

    zipped_pairs = zip(tidy_top_10.index.to_list(),tidy_top_10['value'].to_list())
    binary_masked_cococonet_2 = pd.DataFrame(data = 0, columns = cococonet_2_set_to_zero.columns, index = cococonet_2_set_to_zero.index)
    #binary_masked_cococonet.loc[zip(tuple(tidy_top_10.index.to_list()),tuple(tidy_top_10['value'].to_list()))] = 1
    #binary_masked_cococonet.sum(axis =0)
    for row,column in zipped_pairs:
        binary_masked_cococonet_2.at[row,column] = 1

    ranked_columns_cococonet_1 = double_species_1_trimmed_cococonet.rank()

    dot_product_cococonet_1 = binary_masked_cococonet_2.dot(ranked_columns_cococonet_1)
    subtract_minimum_1 = dot_product_cococonet_1-65 # This is 11+10+9+8+7+6+5+4+3+2


    function_conservations_scores_1 = subtract_minimum_1/(subtract_minimum_1.max().max() - subtract_minimum_1.min().min())
    fc_scores_ranked_1=function_conservations_scores_1.rank()
    sc_scores_1 = fc_scores_ranked_1/fc_scores_ranked_1.shape[0]

    transposed_1 = sc_scores_1.T

    averaged_results = (sc_scores_2+transposed_1)/2
    

    ##process average Rsults to look like table 
    output = pd.DataFrame( columns = [f'{common_name_1} and {common_name_2}'], data = averaged_results.index )
    output['Species 1 Score'] = 'compatability col'
    output['Species 2 Score'] = 'compatability col'
    output['Total Score'] = np.diag(averaged_results)

    return output


    

In [113]:
# trimmed_species_1_cococonet = coconet_species_one[coconet_species_one.columns.intersection(coconet_species_two.columns)]
# trimmed_species_1_cococonet = trimmed_species_1_cococonet[trimmed_species_1_cococonet.index.isin(coconet_species_two.columns)]
# double_species_1_trimmed_cococonet = trimmed_species_1_cococonet.replace(1,0)

# trimmed_species_2_cococonet = coconet_species_two[coconet_species_two.columns.intersection(coconet_species_one.columns)]
# trimmed_species_2_cococonet = trimmed_species_2_cococonet[trimmed_species_2_cococonet.index.isin(coconet_species_one.columns)]
# double_species_2_trimmed_cococonet = trimmed_species_2_cococonet.replace(1,0)
# cococonet_1_set_to_zero = double_species_1_trimmed_cococonet

# cococonet_2_set_to_zero = double_species_2_trimmed_cococonet

# top_10_cococonet_1genes = np.array(
#     [cococonet_1_set_to_zero[c].nlargest(10).index.values for c in cococonet_1_set_to_zero]
# )  # using pair list above, cut down top 10 list to relevant genes, probably by adding list as a column in panda and then filtering panda to index of pair list
# top_10_ccn_1_genes_dataframe = pd.DataFrame(
#     data=top_10_cococonet_1genes,
#     index=cococonet_1_set_to_zero.index,
#     columns=[
#         "One",
#         "Two",
#         "Three",
#         "Four",
#         "Five",
#         "Six",
#         "Seven",
#         "Eight",
#         "Nine",
#         "Ten",
#     ],
# )

# tidy_top_10 = top_10_ccn_1_genes_dataframe.melt(ignore_index= False)

# zipped_pairs = zip(tidy_top_10.index.to_list(),tidy_top_10['value'].to_list())
# binary_masked_cococonet_1 = pd.DataFrame(data = 0, columns = cococonet_1_set_to_zero.columns, index = cococonet_1_set_to_zero.index)
# #binary_masked_cococonet.loc[zip(tuple(tidy_top_10.index.to_list()),tuple(tidy_top_10['value'].to_list()))] = 1
# #binary_masked_cococonet.sum(axis =0)
# for row,column in zipped_pairs:
#     binary_masked_cococonet_1.at[row,column] = 1

# ranked_columns_cococonet_2 = double_species_2_trimmed_cococonet.rank()

# dot_product_cococonet_2 = binary_masked_cococonet_1.dot(ranked_columns_cococonet_2)
# subtract_minimum_2 = dot_product_cococonet_2-65 # This is 11+10+9+8+7+6+5+4+3+2


# function_conservations_scores_2 = subtract_minimum_2/(subtract_minimum_2.max().max() - subtract_minimum_2.min().min())
# fc_scores_ranked_2=function_conservations_scores_2.rank()
# sc_scores_2 = fc_scores_ranked_2/fc_scores_ranked_2.shape[0]




# top_10_cococonet_2genes = np.array(
#     [cococonet_2_set_to_zero[c].nlargest(10).index.values for c in cococonet_2_set_to_zero]
# )  # using pair list above, cut down top 10 list to relevant genes, probably by adding list as a column in panda and then filtering panda to index of pair list
# top_10_ccn_2_genes_dataframe = pd.DataFrame(
#     data=top_10_cococonet_2genes,
#     index=cococonet_2_set_to_zero.index,
#     columns=[
#         "One",
#         "Two",
#         "Three",
#         "Four",
#         "Five",
#         "Six",
#         "Seven",
#         "Eight",
#         "Nine",
#         "Ten",
#     ],
# )

# tidy_top_10 = top_10_ccn_2_genes_dataframe.melt(ignore_index= False)

# zipped_pairs = zip(tidy_top_10.index.to_list(),tidy_top_10['value'].to_list())
# binary_masked_cococonet_2 = pd.DataFrame(data = 0, columns = cococonet_2_set_to_zero.columns, index = cococonet_2_set_to_zero.index)
# #binary_masked_cococonet.loc[zip(tuple(tidy_top_10.index.to_list()),tuple(tidy_top_10['value'].to_list()))] = 1
# #binary_masked_cococonet.sum(axis =0)
# for row,column in zipped_pairs:
#     binary_masked_cococonet_2.at[row,column] = 1

# ranked_columns_cococonet_1 = double_species_1_trimmed_cococonet.rank()

# dot_product_cococonet_1 = binary_masked_cococonet_2.dot(ranked_columns_cococonet_1)
# subtract_minimum_1 = dot_product_cococonet_1-65 # This is 11+10+9+8+7+6+5+4+3+2


# function_conservations_scores_1 = subtract_minimum_1/(subtract_minimum_1.max().max() - subtract_minimum_1.min().min())
# fc_scores_ranked_1=function_conservations_scores_1.rank()
# sc_scores_1 = fc_scores_ranked_1/fc_scores_ranked_1.shape[0]

# transposed_1 = sc_scores_1.T

# averaged_results = (sc_scores_2+transposed_1)/2

In [114]:
averaged_results

,10092at3193,10147at3193,10228at3193,10238at3193,10525at3193,10582at3193,10648at3193,10757at3193,10772at3193,10804at3193,...,9450at3193,9452at3193,9585at3193,9592at3193,9601at3193,9789at3193,9828at3193,9877at3193,9919at3193,9942at3193
10092at3193,0.889823,0.784252,0.953868,0.748501,0.484720,0.851184,0.837951,0.370778,0.969732,0.519594,...,0.435005,0.420785,0.786518,0.570844,0.595920,0.724302,0.398048,0.446154,0.757713,0.945241
10147at3193,0.914936,0.873666,0.966369,0.903897,0.544743,0.985963,0.973863,0.401301,0.947215,0.530048,...,0.519923,0.443303,0.874543,0.848699,0.659672,0.566603,0.658320,0.504277,0.976751,0.637593
10228at3193,0.795365,0.606997,0.871034,0.665960,0.254752,0.731759,0.791782,0.130355,0.784252,0.771750,...,0.583894,0.373483,0.886094,0.736767,0.826802,0.651777,0.595628,0.567481,0.527234,0.825632
10238at3193,0.842667,0.636789,0.671260,0.662195,0.355790,0.831774,0.742652,0.679303,0.548326,0.732709,...,0.830896,0.258737,0.826948,0.752376,0.788858,0.405980,0.698567,0.330494,0.615222,0.204416
10525at3193,0.874653,0.796644,0.873520,0.770288,0.656346,0.865514,0.815543,0.540722,0.986986,0.263781,...,0.313167,0.589925,0.647682,0.724594,0.353560,0.489399,0.335868,0.575742,0.850453,0.839523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9789at3193,0.817298,0.691548,0.849357,0.753071,0.258225,0.841753,0.840547,0.376517,0.527709,0.918117,...,0.817079,0.547083,0.879003,0.896622,0.949335,0.713189,0.961398,0.610762,0.767364,0.553663
9828at3193,0.680436,0.608386,0.797412,0.675245,0.204855,0.855425,0.868402,0.258517,0.487571,0.868548,...,0.756909,0.467100,0.850124,0.970683,0.887630,0.787286,0.934566,0.659746,0.761296,0.536116
9877at3193,0.464834,0.371765,0.719623,0.371472,0.168373,0.418555,0.510711,0.088500,0.414388,0.931386,...,0.595043,0.937052,0.728323,0.829507,0.912049,0.912414,0.956938,0.955768,0.352244,0.819089
9919at3193,0.801470,0.785861,0.901448,0.868548,0.487791,0.984501,0.971048,0.368914,0.906273,0.297119,...,0.374909,0.389421,0.795950,0.900205,0.431021,0.617269,0.692791,0.417495,0.969550,0.556112


In [99]:
fc_scores_ranked_2.shape[0]

6839

In [ ]:

# common_name_1 = Name_resolver.species_name_resolver('rice_jp','common')
# common_name_2 = Name_resolver.species_name_resolver('apple','common')





# ## Read In Cococonets 
# coconet_species_one = coconet_1
# coconet_species_two = coconet_2







# #Identify Pairs for evaluation

# pairs_to_keep = list(set(coconet_species_one.index.to_list()) & set(coconet_species_two.index.to_list()))

# all_pairs_to_evaluate_for_functional_conservation = pd.DataFrame(columns = [f'{common_name_1} and {common_name_2}'],data= pairs_to_keep)


# all_pairs_to_evaluate_for_functional_conservation['Species 1 Score'] = np.nan
# all_pairs_to_evaluate_for_functional_conservation['Species 2 Score'] = np.nan


# ## Trim cococonets to match


# trimmed_species_1_cococonet = coconet_species_one[coconet_species_one.columns.intersection(pairs_to_keep)]
# trimmed_species_1_cococonet = trimmed_species_1_cococonet[trimmed_species_1_cococonet.index.isin(pairs_to_keep)]
# double_species_1_trimmed_cococonet = trimmed_species_1_cococonet.replace(1,0)

# trimmed_species_2_cococonet = coconet_species_two[coconet_species_two.columns.intersection(pairs_to_keep)]
# trimmed_species_2_cococonet = trimmed_species_2_cococonet[trimmed_species_2_cococonet.index.isin(pairs_to_keep)]
# double_species_2_trimmed_cococonet = trimmed_species_2_cococonet.replace(1,0)


In [ ]:
# ## Rank
# species_1_cococonet_ranked = trimmed_species_1_cococonet.rank()
# species_2_cococonet_ranked = trimmed_species_2_cococonet.rank()

# #Do top 10 Genes
# top_10_species_1_genes = np.array(
#     [double_species_1_trimmed_cococonet.T[c].nlargest(10).index.values for c in double_species_1_trimmed_cococonet.T]
# )  # using pair list above, cut down top 10 list to relevant genes, probably by adding list as a column in panda and then filtering panda to index of pair list
# top_10_species_1_genes_dataframe = pd.DataFrame(
#     data=top_10_species_1_genes,
#     index=double_species_1_trimmed_cococonet.index,
#     columns=[
#         "One",
#         "Two",
#         "Three",
#         "Four",
#         "Five",
#         "Six",
#         "Seven",
#         "Eight",
#         "Nine",
#         "Ten",
#     ],
# )

# #Convert 
# top_10_species_1_genes_as_species_2 = top_10_species_1_genes_dataframe

# # Get genes for checking 
# have_species_1_pairs = all_pairs_to_evaluate_for_functional_conservation.loc[all_pairs_to_evaluate_for_functional_conservation[f'{common_name_1} and {common_name_2}'].isin(top_10_species_1_genes_as_species_2.index)]
# trimmed_all_gene_pairs_for_fc = have_species_1_pairs.loc[have_species_1_pairs[f'{common_name_1} and {common_name_2}'].isin(trimmed_species_2_cococonet.index)]
# trimmed_all_gene_pairs_for_fc = trimmed_all_gene_pairs_for_fc.reset_index(drop = True)

# # Get values in species 2 
# for two_genes in trimmed_all_gene_pairs_for_fc.iterrows():
#     current_species_1_gene = two_genes[1][f'{common_name_1} and {common_name_2}']
#     current_species_2_gene = two_genes[1][f'{common_name_1} and {common_name_2}']
#     finger_print_genes = top_10_species_1_genes_as_species_2.loc[current_species_1_gene].to_list()
#     gene_ranks_in_species_2 = species_2_cococonet_ranked.loc[species_2_cococonet_ranked.index.isin(finger_print_genes), current_species_2_gene]
#     avg_rank_in_species_2 = gene_ranks_in_species_2.mean()
#     index_from_pairs = two_genes[0]
#     trimmed_all_gene_pairs_for_fc.at[index_from_pairs, 'Species 1 Score'] = avg_rank_in_species_2

# #Repeat for Species 2 

# top_10_species_2_genes = np.array(
#     [double_species_2_trimmed_cococonet.T[c].nlargest(10).index.values for c in double_species_2_trimmed_cococonet.T]
# )  # using pair list above, cut down top 10 list to relevant genes, probably by adding list as a column in panda and then filtering panda to index of pair list
# top_10_species_2_genes_dataframe = pd.DataFrame(
#     data=top_10_species_2_genes,
#     index=double_species_2_trimmed_cococonet.index,
#     columns=[
#         "One",
#         "Two",
#         "Three",
#         "Four",
#         "Five",
#         "Six",
#         "Seven",
#         "Eight",
#         "Nine",
#         "Ten",
#     ],
# )


# #convert 
# top_10_species_2_genes_as_species_1 = top_10_species_2_genes_dataframe


# # Get values in species 1 
# for two_genes in trimmed_all_gene_pairs_for_fc.iterrows():
#     current_species_1_gene = two_genes[1][f'{common_name_1} and {common_name_2}']
#     current_species_2_gene = two_genes[1][f'{common_name_1} and {common_name_2}']
#     finger_print_genes = top_10_species_2_genes_as_species_1.loc[current_species_2_gene].to_list()
#     gene_ranks_in_species_1 = species_1_cococonet_ranked.loc[species_1_cococonet_ranked.index.isin(finger_print_genes), current_species_1_gene]
#     avg_rank_in_species_1 = gene_ranks_in_species_1.mean()
#     index_from_pairs = two_genes[0]
#     trimmed_all_gene_pairs_for_fc.loc[index_from_pairs, 'Species 2 Score'] = avg_rank_in_species_1

# #Caluclate Divisors 
# Number_of_species_1_genes = len(top_10_species_1_genes_as_species_2)
# Number_of_species_2_genes = len(top_10_species_2_genes_as_species_1)


# species_1_score_divisor = Number_of_species_2_genes - 4.5
# species_2_score_divisor = Number_of_species_1_genes-4.5

# #Divide and Average 
# trimmed_all_gene_pairs_for_fc['Species 1 Score'] = trimmed_all_gene_pairs_for_fc['Species 1 Score']/species_1_score_divisor
# trimmed_all_gene_pairs_for_fc['Species 2 Score'] = trimmed_all_gene_pairs_for_fc['Species 2 Score']/species_2_score_divisor
# trimmed_all_gene_pairs_for_fc['Total Score'] = trimmed_all_gene_pairs_for_fc[['Species 1 Score','Species 2 Score']].mean(axis = 1)

In [110]:
def Calculate_Score_list_for_thresholding(coconet_1, coconet_2, Species_1 = 'rice',Species_2 = 'maize', ):

    import pandas as pd
    import itertools

    common_name_1 = Name_resolver.species_name_resolver(Species_1,'common')
    common_name_2 = Name_resolver.species_name_resolver(Species_2,'common')





    ## Read In Cococonets 
    coconet_species_one = coconet_1
    coconet_species_two = coconet_2







    #Identify Pairs for evaluation

    pairs_to_keep = list(set(coconet_species_one.index.to_list()) & set(coconet_species_two.index.to_list()))

    all_pairs_to_evaluate_for_functional_conservation = pd.DataFrame(columns = [f'{common_name_1} and {common_name_2}'],data= pairs_to_keep)


    all_pairs_to_evaluate_for_functional_conservation['Species 1 Score'] = np.nan
    all_pairs_to_evaluate_for_functional_conservation['Species 2 Score'] = np.nan


    ## Trim cococonets to match


    trimmed_species_1_cococonet = coconet_species_one[coconet_species_one.columns.intersection(pairs_to_keep)]
    trimmed_species_1_cococonet = trimmed_species_1_cococonet[trimmed_species_1_cococonet.index.isin(pairs_to_keep)]
    double_species_1_trimmed_cococonet = trimmed_species_1_cococonet.replace(1,0)

    trimmed_species_2_cococonet = coconet_species_two[coconet_species_two.columns.intersection(pairs_to_keep)]
    trimmed_species_2_cococonet = trimmed_species_2_cococonet[trimmed_species_2_cococonet.index.isin(pairs_to_keep)]
    double_species_2_trimmed_cococonet = trimmed_species_2_cococonet.replace(1,0)



    ## Rank
    species_1_cococonet_ranked = trimmed_species_1_cococonet.rank()
    species_2_cococonet_ranked = trimmed_species_2_cococonet.rank()

    #Do top 10 Genes
    top_10_species_1_genes = np.array(
        [double_species_1_trimmed_cococonet.T[c].nlargest(10).index.values for c in double_species_1_trimmed_cococonet.T]
    )  # using pair list above, cut down top 10 list to relevant genes, probably by adding list as a column in panda and then filtering panda to index of pair list
    top_10_species_1_genes_dataframe = pd.DataFrame(
        data=top_10_species_1_genes,
        index=double_species_1_trimmed_cococonet.index,
        columns=[
            "One",
            "Two",
            "Three",
            "Four",
            "Five",
            "Six",
            "Seven",
            "Eight",
            "Nine",
            "Ten",
        ],
    )

    #Convert 
    top_10_species_1_genes_as_species_2 = top_10_species_1_genes_dataframe

    # Get genes for checking 
    have_species_1_pairs = all_pairs_to_evaluate_for_functional_conservation.loc[all_pairs_to_evaluate_for_functional_conservation[f'{common_name_1} and {common_name_2}'].isin(top_10_species_1_genes_as_species_2.index)]
    trimmed_all_gene_pairs_for_fc = have_species_1_pairs.loc[have_species_1_pairs[f'{common_name_1} and {common_name_2}'].isin(trimmed_species_2_cococonet.index)]
    trimmed_all_gene_pairs_for_fc = trimmed_all_gene_pairs_for_fc.reset_index(drop = True)

    # Get values in species 2 
    for two_genes in trimmed_all_gene_pairs_for_fc.iterrows():
        current_species_1_gene = two_genes[1][f'{common_name_1} and {common_name_2}']
        current_species_2_gene = two_genes[1][f'{common_name_1} and {common_name_2}']
        finger_print_genes = top_10_species_1_genes_as_species_2.loc[current_species_1_gene].to_list()
        gene_ranks_in_species_2 = species_2_cococonet_ranked.loc[species_2_cococonet_ranked.index.isin(finger_print_genes), current_species_2_gene]
        avg_rank_in_species_2 = gene_ranks_in_species_2.mean()
        index_from_pairs = two_genes[0]
        trimmed_all_gene_pairs_for_fc.at[index_from_pairs, 'Species 1 Score'] = avg_rank_in_species_2

    #Repeat for Species 2 

    top_10_species_2_genes = np.array(
        [double_species_2_trimmed_cococonet.T[c].nlargest(10).index.values for c in double_species_2_trimmed_cococonet.T]
    )  # using pair list above, cut down top 10 list to relevant genes, probably by adding list as a column in panda and then filtering panda to index of pair list
    top_10_species_2_genes_dataframe = pd.DataFrame(
        data=top_10_species_2_genes,
        index=double_species_2_trimmed_cococonet.index,
        columns=[
            "One",
            "Two",
            "Three",
            "Four",
            "Five",
            "Six",
            "Seven",
            "Eight",
            "Nine",
            "Ten",
        ],
    )


    #convert 
    top_10_species_2_genes_as_species_1 = top_10_species_2_genes_dataframe


    # Get values in species 1 
    for two_genes in trimmed_all_gene_pairs_for_fc.iterrows():
        current_species_1_gene = two_genes[1][f'{common_name_1} and {common_name_2}']
        current_species_2_gene = two_genes[1][f'{common_name_1} and {common_name_2}']
        finger_print_genes = top_10_species_2_genes_as_species_1.loc[current_species_2_gene].to_list()
        gene_ranks_in_species_1 = species_1_cococonet_ranked.loc[species_1_cococonet_ranked.index.isin(finger_print_genes), current_species_1_gene]
        avg_rank_in_species_1 = gene_ranks_in_species_1.mean()
        index_from_pairs = two_genes[0]
        trimmed_all_gene_pairs_for_fc.loc[index_from_pairs, 'Species 2 Score'] = avg_rank_in_species_1

    #Caluclate Divisors 
    Number_of_species_1_genes = len(top_10_species_1_genes_as_species_2)
    Number_of_species_2_genes = len(top_10_species_2_genes_as_species_1)


    species_1_score_divisor = Number_of_species_2_genes - 4.5
    species_2_score_divisor = Number_of_species_1_genes-4.5

    #Divide and Average 
    trimmed_all_gene_pairs_for_fc['Species 1 Score'] = trimmed_all_gene_pairs_for_fc['Species 1 Score']/species_1_score_divisor
    trimmed_all_gene_pairs_for_fc['Species 2 Score'] = trimmed_all_gene_pairs_for_fc['Species 2 Score']/species_2_score_divisor
    trimmed_all_gene_pairs_for_fc['Total Score'] = trimmed_all_gene_pairs_for_fc[['Species 1 Score','Species 2 Score']].mean(axis = 1)
        
    return trimmed_all_gene_pairs_for_fc

In [111]:
Calculate_Score_list_for_thresholding(coconet_species_one,coconet_species_two,'rice_jp','soybean')

,rice_jp and soybean,Species 1 Score,Species 2 Score,Total Score
0,197438at3193,0.874636,0.746448,0.810542
1,292029at3193,0.814266,0.875646,0.844956
2,114093at3193,0.825181,0.772888,0.799034
3,132299at3193,0.842651,0.908494,0.875572
4,885620at3193,0.858878,0.926915,0.892896
...,...,...,...,...
6834,132570at3193,0.850582,0.773926,0.812254
6835,124155at3193,0.987622,0.951525,0.969573
6836,778072at3193,0.693525,0.654678,0.674102
6837,126160at3193,0.989202,0.982208,0.985705


In [120]:
generate_sc_table_formatted(coconet_species_one,coconet_species_two,'rice_jp','soybean')

,rice_jp and soybean,Species 1 Score,Species 2 Score,Total Score
0,10092at3193,compatability col,compatability col,0.889823
1,10147at3193,compatability col,compatability col,0.873666
2,10228at3193,compatability col,compatability col,0.871034
3,10238at3193,compatability col,compatability col,0.662195
4,10525at3193,compatability col,compatability col,0.656346
...,...,...,...,...
6834,9789at3193,compatability col,compatability col,0.713189
6835,9828at3193,compatability col,compatability col,0.934566
6836,9877at3193,compatability col,compatability col,0.955768
6837,9919at3193,compatability col,compatability col,0.969550


In [122]:
species_to_run_on = pd.read_csv('/data/passala/Generated_Tables/Reference_tables/species_for_running_cross_ortho_analysis.csv')


In [123]:
list_of_species_to_run_on = species_to_run_on['Common Name'].to_list()


In [124]:
all_coconets = {}
for species in tqdm.tqdm(list_of_species_to_run_on):
    current_net = pickle.load( open( f"/data/passala/Generated_Tables/Comparing_all_orthogorups_across_plants/Orthogroup_based_coconets/{species}_merged_cococonet_pickle.p", "rb" ) )
    all_coconets[species] = current_net

100%|██████████| 15/15 [00:07<00:00,  2.10it/s]


In [125]:
all_combos_to_run_on = list(itertools.combinations(list_of_species_to_run_on,2))


In [126]:
len(all_combos_to_run_on)


105

In [127]:
generated_files = os.listdir('/data/passala/Generated_Tables/Comparing_all_orthogorups_across_plants/1_1_ortho_FC_tables')

In [ ]:
len(generated_files)

In [128]:
for combo in tqdm.tqdm(all_combos_to_run_on):
    species_1 = combo[0]
    species_2 = combo[1]
    coconet_spec_1 = all_coconets[species_1]
    coconet_spec_2 = all_coconets[species_2]
    
    score_list = generate_sc_table_formatted(coconet_spec_1, coconet_spec_2, species_1,species_2)
    score_list.to_csv(f'/data/passala/Generated_Tables/Comparing_all_orthogorups_across_plants/1_1_ortho_SC_tables/{species_1}_{species_2}_NM.csv')


100%|██████████| 105/105 [1:43:28<00:00, 59.13s/it]
